# Statistical birds eye view of the contents in an AiiDAdb

This is the first of two deliverable for the SiSc-Lab2020 project.

Authors = 

Supervisors: Jens Bröder, Dr. Daniel Wortmann, Johannes Wasmer, Prof. Dr. Stefan Blügel.

# Instructions by supervisors

## Jens

You have to implement this notebook.

In the end only text (markdown) cells and output results of code cells should be seen if one hides the code cells (hide_code extension).

That can easily exported into a nice pdf file (google it, probably will find sth with `nbconvert`).

Also the amount of python code in this notebook should be minimal.

Rather, export the functions you use to python file(s) and import them here (hide complexity).

Optional dump query results in a file, from which results will be reread for speed, i.e cache results.

## Johannes



After talking with Jens about it, here are some additional thoughts on the code structure and implementation, for both deliverables.

The **primary** goal is of course that the code should work, produce nice output (and helpful error messages), obviously.

The **secondary** goal is speed. How long do you expect your code to run on a dataset of a given size? Are there multiple paths to a goal, but with differing performance?

You can break the runtime down into several steps: data acquisition, data transformation (or preprocessing), data analysis, data visualization. In this project, we will rename/replace these steps to: **querying, de-/serialization, analysis=visualization**.

**Querying the database.** Performance considerations:
- Performance measurement: use the magics `%time` and `%timeit`.
- Query evaluations: queries (in general) use 'lazy evaluation'.
  - *Query building* methods build the query but do not execute it. These are chainable methods like `append()`, `get_outgoing()`, etc.
  - *Query execution* methods send the query to the database to be evaluated. There are two kinds:
    - non-iterator methods: e.g. `all()`, `first()`, etc. These return a result `list`: all items are loaded into memory.
    - iterator methods: e.g. `iterall()`, `iterdict()`. These return a result `Generator`: only one item at a time is loaded into memory.
    
**De-/serialization**, i.e. writing and reading it to/from a file. *Keep in mind: if you come to the conclusion this is unnecessary, then justify it!* Considerations:
- Necessity: we assume 'yes'. So you need serialization/deserialization routine(s).
- Code design: we recommend to write a serializer that moves *all* data needed from aiida to file (perform query & serialization). Then the visualization methods are decoupled from aiida and load data from that file. Advantages: a) only needs to be called when data in database changed, b) similar queries for different visualizations can be performed only once. One design option is this:
  ```python
  serialize = sisclab.Serializer(profile)
  serialize.to_file(filepath)
  visualize = sisclab.Visualizer(filepath)
  visualize.histogram(cumulative=True, plot_options)
  # plots histogram
  ```
- Serialization format: there are two practical options (maybe more):
  - `dict`: tree-like. JSON format. One `dict` per file. choose key-value (nested?) based on use-case. in general, `uuid` is a good key.
  - `pandas.Dataframe`: could be preferrable in some cases.
- Serialization location:
  - one file or several files?
  - we recommend to de/serialize from/to `sisclab/data/` folder. It is included in the project's `.gitignore` file, so nothing in it gets committed to/from git (git is for code, not for data; the code generates the data).
- Transformation:
  - if needed, decide where to put needed data transformations (before serialization or after deserialization) to minimize them.
- Deserialization: 
  - a class (as above) might help to define the deserialization format only once for all visualization methods.
  

**Visualization**:
- Prefer `bokeh` to `matplotlib` or other libs wherever possible, unless you have a good justification.
- In `D1`, static plots are okay, interactive plots are a bonus.
- Lists results (when plot is overkill) will look nicer in a notebook if they are a `pandas.Series` or `pandas.Dataframes`.
- Think about function signatures. Can you generalize them to make a nice interface? For example, a signature for SubtaskD1.c might look like this:
  ```python
  def node_type_summary(user_list : list = [], node_basetype : Node = Data,
                        chart_type : bokeh.chart_type = bokeh.pie_chart, plot : bool = True):
    """
    :param user_list: list of users. empty list = all users = default.
    :param node_basetype: subdivides chart into subtypes. Valid base type examples: ProcessNode, CalculationNode, WorkflowNode, Data, ArrayData.
    :param chart_type: bokeh visualization type. pie chart = default.
    :param plot: True: show plot, don't return data. False: don't plot, return data.
    :return: stats: a dictionary {node_subtype : node_count}, insertion-order sorted in descending order.
    :rtype: dict.
    """
  ```


# Imports

In [2]:
# magics:
# # autoreload imports. 
# # intent: if i change sth in import, i don't have to restart kernel. enable only for development.
%load_ext autoreload
%autoreload 2
# # choose matplotlib backend. backend 'notebook' allows interactive plots if your env allows it.
%matplotlib notebook


In [3]:
# python imports:
from collections import Counter
import time
#from pprint import pprint

In [4]:
#%pylab inline
#figuresize=(18, 4)
from collections import Counter
from math import pi
import pandas as pd
from bokeh.io import output_file,output_notebook, show
from bokeh.layouts import column
from bokeh.palettes import Category20c
from bokeh.plotting import figure
from bokeh.transform import cumsum
from bokeh.models import Legend,LegendItem

In [5]:
# aiida imports:
from aiida import load_profile
profile = load_profile()

# ggf add futher imports
from aiida.orm import QueryBuilder as QB
from aiida.orm import Node, User, CalcJobNode, Computer, Code
from aiida.plugins import DataFactory

from aiida.common.constants import elements as PeriodicTableElements

In [6]:
# project imports:
import helpers

# equivalent ('.' is the sisc_lab directory):
# from . import helpers
# alternative:
# from .helpers import print_bold
# from .helpers import * ('*' import everything; use of '*' is considered bad style)

In [7]:
# (example:)
helpers.print_bold(f"This notebook/dashboard will visualize the contents from the database of profile {profile.name}")

This notebook/dashboard will visualize the contents from the database of profile generic


# SubtaskD1.a: Node information

Task:

In [7]:
# query for all nodes
print('Information on nodes in the DB: \n')
now = time.strftime("%c")
print('last executed on {}'.format(now))
q = QB()
q.append(Node, project=['id', 'ctime', 'mtime', 'node_type'], tag='node')
q.append(User, with_node='node', project='email')
# TODO: execute query here
t = time.time()
elapsed = time.time() - t
res = q.all()
totalnodes = len(res)
print("Total number of nodes in the database: {} (retrieved in {} s.)".format(totalnodes, elapsed))

Information on nodes in the DB: 

last executed on Wed Jan  6 12:05:19 2021
Total number of nodes in the database: 7792 (retrieved in 0.00016808509826660156 s.)


# SubtaskD1.b: Users

Task: print out a list of Users and how many nodes belong to them

for example

```
Users:
- j.broeder@fz-juelich.de created 182 nodes
- tests@aiida.mail created 104 nodes
```

In [13]:
users = Counter([r[4] for r in res])
print("Users:")
for count, email in sorted((v, k) for k, v in users.items())[::-1]:
    print("* {} created {} nodes".format(email, count))

Users:
* j.broeder@fz-juelich.de created 5629 nodes
* nicolas.mounet@epfl.ch created 1981 nodes
* gianluca.prandini@epfl.ch created 93 nodes
* aiida@localhost created 86 nodes
* antimo.marrazzo@epfl.ch created 3 nodes


# SubtaskD1.c: Node types

Task: plot node information in two pie chart plots

One showing what data nodes there (with their lowest class names(node_type)) I.e Dict, K-pointsData, CifData, FleurinpData...

And one chart showning the process nodes, (with their lowest class names(process_type) i.e CalcjobNodes: FleurCalcjob, FleurinputgenCalcjob, ...

WorkChain nodes: FleurSCFWorkchain, FleurBandDosWorkchain, ..., calcfunctions, and workfunction nodes are fine to not show the lowest class names


In [14]:
#node types
types = Counter([r[3] for r in res])
print("Node types:")

for count, typestring in sorted((v, k) for k, v in types.items())[::-1]:
    print("* {}: {} nodes".format(typestring, count))

Node types:
* data.structure.StructureData.: 5511 nodes
* data.dict.Dict.: 707 nodes
* data.array.kpoints.KpointsData.: 342 nodes
* data.upf.UpfData.: 264 nodes
* process.calculation.calcjob.CalcJobNode.: 214 nodes
* data.remote.RemoteData.: 214 nodes
* data.folder.FolderData.: 214 nodes
* data.array.ArrayData.: 171 nodes
* process.workflow.workfunction.WorkFunctionNode.: 49 nodes
* data.fleur.fleurinp.FleurinpData.: 42 nodes
* data.code.Code.: 36 nodes
* process.workflow.workchain.WorkChainNode.: 28 nodes


In [15]:
#split data nodes and process nodes
labelst_1,labelst_2=[],[]
sizest_1,sizest_2=[],[]
#labelst = [label.split('.')[0]=='data' for label in types.keys()]
#sizest = [nnodes for nnodes in types.values()]
for k,v in types.items():
    if k.split('.')[0]=='data':
        labelst_1.append(k.split('.')[-2])
        sizest_1.append(v)
    elif k.split('.')[0]=='process':
        labelst_2.append(k.split('.')[-2])
        sizest_2.append(v)
        
#plot data nodes
#output_file("pie.html")
output_notebook()
x = dict(zip(labelst_1,sizest_1)) 
data=pd.DataFrame.from_dict(dict(x),orient='index').reset_index().rename(index=str,columns={0:'value','index':'data_nodes'})
data['angle'] = data['value']/sum(x.values()) * 2*pi
data['color'] = Category20c[len(x)]
p = figure(plot_height=350, title="Data Nodes", toolbar_location=None,
           tools="hover", tooltips="@data_nodes: @value")
p.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='data_nodes', source=data)
p.axis.axis_label=None
p.axis.visible=False
p.grid.grid_line_color = None 
#show(p)                        

#plot process node

x1 = dict(zip(labelst_2,sizest_2)) 
data=pd.DataFrame.from_dict(dict(x1),orient='index').reset_index().rename(index=str,columns={0:'value','index':'process_nodes'})
data['angle'] = data['value']/sum(list(x1.values())) * 2*pi
data['color'] = Category20c[len(x1)]
p1 = figure(plot_height=350, title="Process Nodes", toolbar_location=None,
           tools="hover", tooltips="@process_nodes: @value")
p1.wedge(x=0, y=1, radius=0.4,
        start_angle=cumsum('angle', include_zero=True), end_angle=cumsum('angle'),
        line_color="white", fill_color='color', legend_field='process_nodes', source=data)
p1.axis.axis_label=None
p1.axis.visible=False
p1.grid.grid_line_color = None 
show(column(p,p1))


Loading BokehJS ...

# SubtaskD1.d: Histogram

Task: Cumulative Histogram/ or line plot by ctime & mtime of all nodes over time

In [31]:
# line plot by ctime & mtime

ctimes = sorted(r[1] for r in res)
mtimes = sorted(r[2] for r in res)
num_nodes_integrated = range(len(ctimes))
df = pd.DataFrame({'A':ctimes,"B":mtimes})

#print(df.head())
#df = pd.DataFrame({'A':np.random.randn(100).cumsum(),"B":np.random.randn(100).cumsum()})

#plot multiline
p = figure(plot_width=900, plot_height=300, x_axis_type='datetime')
r=p.multi_line([df['A'], df['B']],  
               [df.index, df.index],   
               color=["firebrick", "navy"],   
               alpha=[0.8, 0.6],     
               line_width=[2,1],     
              )

legend=Legend(items=[
    LegendItem(label="ctime",renderers=[r],index=0),
    LegendItem(label="mtime",renderers=[r],index=1),
])
p.add_layout(legend)
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Number of nodes'

show(p)

# SubtaskD1.e: Codes

Task: List Code names, sorted by by how many calcjobs where run with each

In [86]:
codes = Code.objects.all()
result = {code.label : len(code.get_outgoing(node_class=CalcJobNode).all_nodes()) for code in codes}
result_df=pd.Series(result).sort_values(ascending=False)
result_df

pw-SVN-piz-daint           171
fleur_mpi_v0.28             22
inpgen                      21
yambo-4.5.1-yambo            0
yambo-4.5.1-p2y              0
inpgen_v0.28                 0
inpgen_iff_0.28              0
fleur_iff_0.28               0
fleur_mpi_max_2              0
fleur_dev_mpi                0
fleur_max_2                  0
inpgen_max_2                 0
fleur_dev_mpi_booster        0
fleur_mac_3_mpi              0
fleur_max_1.3_dev            0
fleur_mpi_max_1.3_dev        0
wannier90-3.1.0-wannier      0
cp2k-7.1                     0
siesta-v4.1-rc1-siesta       0
fleur-0.30-inpgen            0
fleur-0.30-fleur_MPI         0
qe-6.5-matdyn                0
qe-6.5-dos                   0
qe-6.5-q2r                   0
qe-6.5-pw2wannier90          0
qe-6.5-projwfc               0
qe-6.5-neb                   0
qe-6.5-ph                    0
qe-6.5-pp                    0
qe-6.5-cp                    0
qe-6.5-pw                    0
fleur_v0.28                  0
dtype: i

# SubtaskD1.f: Groups

Task: List all group names with how many nodes they contain (verdi group list -C) (exclude import and export groups)

In [57]:
def ListGroup(Group : list, exclude: list=[]):
    """ return the group names and nodes they contain
    
    :the list of all groups
    :the list of all excluded groups name
    """
    print('{:<52}{:6}'.format('Group names:','sizes:'))
    for a in Group:
        flag=0
        type = a[0].type_string
        for ex in exclude:
            if ex in type:
                flag=1
        if(flag):
            continue     
        else:
            ## the line below contains all the properties
            ##print(a[0].label,' ',a[0].user,' ',a[0].type_string,' ',a[0].description)

            print('{:<50}|{:5}'.format(a[0].label,len(a[0].nodes)))

In [58]:
load_profile()
#!verdi group list --all
from aiida.orm import QueryBuilder
from aiida.orm import load_node, Node, Group, Computer, User, CalcJobNode, Code, StructureData

qb = QueryBuilder()
qb.append(Group)
#print(qb.all())

group = qb.all()
ListGroup(group,exclude=['export','import'])

Group names:                                        sizes:
SSSP_1.1_efficiency                               |   85
SSSP_1.1_precision                                |   85
sg15-oncv-1.1                                     |   69
codes                                             |   18
fusion_materials                                  |   79
stable_binary_metals_from_materialsproject        | 5058
fleur_initial_cls_finished                        |    7
delta_parameters_gutstav_soc                      |   71
fusion_strucs_refinded                            |   23


# SubtaskD1.g: Structures

Task: Further analyze what structures are in the DB

Number of structureData node versus how many atoms they contain. 

here interactive with bokeh hover tool showing the structure formula and uuid

Number of StructureData nodes versus elements bokeh bar chart, since there are over 
100 elements in the periodic table you can split it over several plots, or just use the charge number as in 
'example/element_content.png' but then make it interactive that once one hovers 
with the mouse over a bar it tells you what element it is and how many structures there are containing this element-

In [79]:
from aiida.orm import QueryBuilder
from aiida.orm import load_node, Node, Group, Computer, User, CalcJobNode, Code, StructureData
from HelperPackage import DataProcessing
from DataProcessing.DataVisu import StrucDataForm

def NumStructureNode():
    #### return the pd.DataFrame including elements and number of each element
    import pandas as pd
    StrucList = []
    qb = QueryBuilder()
    qb.append(StructureData)
    print('number of StructureData Nodes:',qb.count())
    qb.count()

    for struc, in qb.all()[:]:
        form = struc.get_formula()
        struct = StrucDataForm(form)
        StrucList = StrucList+ [struct.FormAnalyse()]
    return pd.DataFrame(StrucList).fillna(0)

def ShowElements(Data):
    from bokeh.io import output_file, show
    from bokeh.models import ColumnDataSource,HoverTool
    from bokeh.plotting import figure
    from bokeh.io import output_notebook
    from bokeh.palettes import inferno

    output_file("ShowingElements.html")
    #data = NumStructureNode()
    data = Data
    elements = list(data.columns)
    counts = list(data.astype(bool).sum(axis=0))
    #print(counts)
    #print(elements)
    
    source = ColumnDataSource(data=dict(elements=elements, counts=counts,color=inferno(len(elements))))
    
    TOOLTIPS = [
    ("element", "@elements"),
    ("(x,y)", "($x, $y)"),
    ("Number of Structures containing this element", "@counts"),
    ]

    p = figure( y_range=elements,x_range=(0,500), plot_width=800, plot_height=800, title="Elements Counts",tools = [HoverTool(mode='hline')], tooltips=TOOLTIPS)
    #print('step figure done')
    p.hbar(y="elements", right="counts", height=0.5, left=0, color='color',  source=source)
    #print('step hbar done')
    
    output_notebook()
    p.xgrid.grid_line_color = None
    #p.legend = False
    show(p)
    
    
def ShowFormula(Data):
    from bokeh.io import output_file, show
    from bokeh.models import ColumnDataSource,HoverTool
    from bokeh.plotting import figure
    from bokeh.io import output_notebook
    from bokeh.palettes import inferno

    data = Data
    elements = list(data.keys())
    counts = list(len(data[key])/2 for key in data.keys())
    formulas = list(data[key] for key in data.keys())

    length = len(elements)
    source = ColumnDataSource(data=dict(elements=elements, counts=counts,formulas=formulas,color=inferno(length)))
    
    TOOLTIPS = [
    ("Number of Atoms", "@elements"),
    ("(x,y)", "($x, $y)"),
    ("Number of Nodes", "@counts"),
    ("id and formula", "@formulas"),
    ]

    p = figure( y_range=(0,160),x_range=(0,1000), plot_width=800, plot_height=800, title="Atoms Count",tools = [HoverTool(mode='hline')], tooltips=TOOLTIPS)
    #print('step figure done')
    p.hbar(y="elements", right="counts", height=0.5, left=0, color='color',  source=source)
    #print('step hbar done')
    
    output_notebook()
    p.xgrid.grid_line_color = None
    #p.legend = False
    show(p)
    
def GetFormulaDict():
    ## may be not necessary, return the formulas and nodes containing this formula
    import numpy as np
    from aiida.orm import QueryBuilder,StructureData
    qb = QueryBuilder()
    qb.append(StructureData)
    Mydict = qb.all()
    Mydict = np.ravel(Mydict)

    Newdict = {}
    for dict in Mydict:
        if dict.get_formula in Newdict.keys():
            Newdict[dict.get_formula()].append(dict.uuid)
        else:
            Newdict[dict.get_formula()] = [dict.uuid]
    return Newdict

def AtomsNumNodes():
    from aiida.orm import QueryBuilder,StructureData
    import numpy as np
    qb = QueryBuilder()
    qb.append(StructureData)
    StructDatas = qb.all()
    Newdict = {}
    
    for data, in StructDatas:
        CompositionDict = data.get_composition()
        NumAtom = np.sum(list(CompositionDict.values()))
        if NumAtom in Newdict.keys():
            Newdict[NumAtom].append(data.uuid)
            Newdict[NumAtom].append(data.get_formula())
        else:
            Newdict[NumAtom] = [data.uuid,data.get_formula()]
    return Newdict

In [80]:
dic = AtomsNumNodes()
ShowFormula(dic)

Loading BokehJS ...

In [81]:
data = NumStructureNode()
ShowElements(data)

number of StructureData Nodes: 5169


Loading BokehJS ...

In [82]:
# initialize the Structure data
from aiida.orm import QueryBuilder
from aiida.orm import load_node, Node, Group, Computer, User, CalcJobNode, Code, StructureData
from HelperPackage import DataProcessing
from DataProcessing.DataVisu import StrucDataForm

qb = QueryBuilder()
qb.append(StructureData)

structures = qb.all()

for structure in structures[:]:
    formula = structure[0].get_formula()
    struct = StrucDataForm(formula)
    structure[0].get_composition()
    print(struct.FormAnalyse())

{'Mg': '16', 'Zn': '80'}
{'Cd': '3', 'Zr': '1'}
{'Nd': '2', 'Te': '4'}
{'As': '4', 'Np': '2'}
{'Co': '2', 'O': '2'}
{'Ba': '10', 'Pb': '6'}
{'Ca': '2', 'Sb': '4'}
{'As': '24', 'Nb': '40'}
{'S': '8', 'V': '4'}
{'B': '20', 'Gd': '8'}
{'Co': '2', 'Y': '2'}
{'O': '2', 'Ti': '12'}
{'Na': '4', 'O': '8'}
{'Se': '40', 'V': '23'}
{'Cd': '2', 'Li': '2'}
{'Fe': '5', 'Y': '1'}
{'C': '12', 'Mn': '28'}
{'Lu': '10', 'Rh': '6'}
{'Ag': '2', 'Nd': '1'}
{'S': '32', 'Tb': '22'}
{'Cu': '2', 'Te': '2'}
{'Cl': '24', 'Mo': '6'}
{'Pu': '1', 'Si': '2'}
{'As': '12', 'V': '16'}
{'Pb': '6', 'Zr': '10'}
{'Au': '2', 'Nb': '1'}
{'Hf': '5', 'Sc': '1'}
{'Fe': '4', 'Sb': '12'}
{'Cu': '6', 'Pm': '2'}
{'In': '3', 'Mg': '1'}
{'Eu': '2', 'Pb': '2'}
{'K': '16', 'Tl': '22'}
{'Pt': '5', 'Sr': '1'}
{'Ag': '20', 'Sr': '20'}
{'Fe': '29', 'Nd': '3'}
{'In': '2', 'Sc': '6'}
{'Ru': '6', 'V': '2'}
{'Hf': '1', 'Zn': '1'}
{'K': '4', 'O': '8'}
{'Lu': '1', 'S': '1'}
{'Al': '4', 'Nd': '2'}
{'O': '3', 'Pa': '1'}
{'Re': '8', 'U': '4'}
{'U': 

{'Ni': '2', 'Se': '2'}
{'Mn': '2', 'Se': '2'}
{'O': '4', 'Ti': '3'}
{'Nd': '6', 'Yb': '2'}
{'C': '15', 'V': '18'}
{'Lu': '6', 'Pr': '2'}
{'Co': '7', 'Nb': '6'}
{'Ir': '2', 'Nb': '2'}
{'Gd': '2', 'Ni': '4'}
{'Si': '1', 'Te': '2'}
{'Fe': '2', 'Zr': '6'}
{'Sn': '6', 'Th': '10'}
{'Nb': '1', 'S': '2'}
{'Dy': '22', 'S': '32'}
{'Au': '5', 'Eu': '1'}
{'Dy': '3', 'Nd': '1'}
{'Ni': '12', 'P': '5'}
{'Se': '12', 'Ti': '24'}
{'Cl': '6', 'Ti': '2'}
{'Fe': '4', 'N': '1'}
{'Cu': '1', 'Dy': '1'}
{'Re': '6', 'Ru': '2'}
{'Ba': '2', 'Cu': '26'}
{'Ag': '4', 'Eu': '6'}
{'Cd': '2', 'Th': '1'}
{'P': '10', 'Pr': '2'}
{'Na': '6', 'Pt': '2'}
{'Cd': '1', 'Zr': '2'}
{'Be': '2'}
{'Dy': '3', 'Sm': '1'}
{'Nb': '1', 'S': '2'}
{'Gd': '1', 'P': '1'}
{'Co': '17', 'Th': '2'}
{'In': '1', 'Pd': '1'}
{'Si': '2', 'Th': '1'}
{'Co': '4', 'S': '8'}
{'Dy': '4', 'Zn': '12'}
{'Cd': '6', 'Y': '2'}
{'Ag': '4', 'Ba': '4'}
{'S': '16', 'V': '10'}
{'In': '1', 'Sm': '1'}
{'Na': '1', 'Rb': '3'}
{'Ca': '2', 'Ge': '4'}
{'Cs': '2', 'Sn': '6'}

{'Au': '3', 'Nd': '1'}
{'C': '2', 'Pa': '1'}
{'Pr': '1', 'Te': '1'}
{'Ga': '3', 'Ni': '2'}
{'La': '1', 'Ni': '5'}
{'Re': '6', 'Tc': '2'}
{'Pt': '12', 'Tl': '8'}
{'Sn': '8', 'Th': '10'}
{'Si': '2', 'U': '6'}
{'Ge': '5', 'Pr': '1'}
{'In': '4', 'Te': '4'}
{'Ca': '2', 'Ni': '4'}
{'Be': '8', 'Fe': '4'}
{'Bi': '2', 'O': '4'}
{'Os': '1', 'V': '1'}
{'Ba': '2', 'Hg': '4'}
{'Ge': '2', 'Hf': '4'}
{'Hg': '4', 'Sr': '2'}
{'Br': '2', 'La': '2'}
{'La': '3', 'Pb': '1'}
{'Hf': '4', 'Si': '4'}
{'C': '5', 'Nb': '6'}
{'Mn': '4', 'Zr': '2'}
{'La': '8', 'Sb': '6'}
{'O': '1', 'Ti': '2'}
{'La': '6', 'Rh': '18'}
{'Cd': '2', 'Er': '1'}
{'As': '14', 'Re': '6'}
{'Cr': '2', 'P': '8'}
{'As': '1', 'Y': '1'}
{'Ni': '6', 'P': '3'}
{'O': '26', 'Ti': '14'}
{'As': '2', 'Cr': '2'}
{'Ag': '10', 'Cd': '16'}
{'Bi': '4', 'Cs': '2'}
{'Al': '44', 'Mo': '10'}
{'Pd': '2', 'Tb': '2'}
{'Gd': '1', 'Zn': '1'}
{'Hf': '1', 'Rh': '3'}
{'Mg': '76', 'Sr': '18'}
{'Tb': '1', 'Tl': '3'}
{'Br': '6', 'Np': '2'}
{'C': '1', 'Tb': '2'}
{'Ga': '8'

{'Al': '1', 'Cu': '3'}
{'Dy': '1', 'P': '1'}
{'Sm': '3', 'Yb': '1'}
{'Au': '2', 'Sc': '1'}
{'Nd': '4', 'Sb': '2'}
{'I': '2', 'Tm': '1'}
{'O': '8', 'Pt': '6'}
{'Mg': '2', 'Pb': '1'}
{'Fe': '3', 'Mn': '1'}
{'Cr': '1', 'Se': '2'}
{'Er': '2', 'Sm': '6'}
{'Bi': '1', 'Ga': '1'}
{'As': '32', 'Cd': '48'}
{'Ir': '1', 'Tm': '1'}
{'Ho': '1', 'Nd': '3'}
{'Al': '1', 'Ce': '3'}
{'Sn': '6', 'Ti': '10'}
{'F': '9', 'Pd': '3'}
{'Sb': '1', 'Th': '1'}
{'Ce': '1', 'Sn': '3'}
{'Cu': '4', 'Ho': '2'}
{'In': '8', 'Ir': '4'}
{'La': '4', 'Sb': '2'}
{'Gd': '6', 'H': '18'}
{'B': '4', 'Mn': '3'}
{'Ge': '12', 'Pt': '8'}
{'Cr': '4', 'Hf': '2'}
{'Fe': '4', 'Tm': '2'}
{'Cd': '6', 'Th': '2'}
{'Na': '3', 'Rb': '1'}
{'Ca': '2', 'Ga': '2'}
{'Mn': '12', 'Nd': '1'}
{'Ce': '2', 'Mg': '4'}
{'Ce': '2', 'Se': '4'}
{'Al': '3', 'Zr': '4'}
{'Co': '6', 'Nb': '2'}
{'Pd': '2', 'Sb': '2'}
{'Au': '6', 'Mg': '2'}
{'B': '2', 'Tb': '1'}
{'Be': '27', 'Ti': '9'}
{'Co': '2', 'Zr': '4'}
{'O': '2', 'Pu': '1'}
{'Hg': '6', 'Tb': '2'}
{'Mg': '1', 

{'Ni': '6', 'Pm': '2'}
{'Au': '1', 'Hf': '2'}
{'Be': '26', 'Zr': '2'}
{'Eu': '2', 'O': '3'}
{'O': '14', 'U': '8'}
{'Ce': '6', 'O': '9'}
{'Fe': '8', 'S': '10'}
{'Sc': '2', 'Sm': '6'}
{'Bi': '2', 'Sr': '4'}
{'Ge': '1', 'Ni': '3'}
{'Cl': '8', 'Sm': '4'}
{'B': '2', 'Np': '1'}
{'Er': '6', 'Zn': '23'}
{'Al': '1', 'Tc': '2'}
{'Pt': '12', 'Si': '5'}
{'Bi': '1', 'Na': '3'}
{'As': '26', 'Ba': '40'}
{'Cr': '1', 'Ni': '2'}
{'B': '4', 'Ni': '12'}
{'Cu': '1', 'Li': '3'}
{'Hf': '2', 'Hg': '1'}
{'Si': '6', 'Zr': '10'}
{'Ru': '6', 'W': '2'}
{'Au': '6', 'Cd': '2'}
{'Mg': '2', 'Pd': '6'}
{'As': '6', 'Eu': '8'}
{'Li': '1', 'Mg': '2'}
{'Dy': '4', 'Si': '4'}
{'Li': '1', 'Tl': '1'}
{'Pt': '4', 'Si': '4'}
{'Eu': '1', 'Si': '2'}
{'Br': '32', 'Os': '8'}
{'As': '4', 'Cu': '10'}
{'Bi': '4', 'U': '2'}
{'Cu': '21', 'Pd': '7'}
{'Bi': '18', 'I': '4'}
{'Li': '3', 'Mg': '3'}
{'Au': '1', 'Ho': '1'}
{'Ni': '6', 'Ta': '2'}
{'Be': '8', 'W': '4'}
{'I': '2', 'Nd': '1'}
{'Re': '8', 'U': '4'}
{'Be': '26', 'Ca': '2'}
{'Ni': '4'

...

{'Pt': '3', 'Tc': '1'}
{'Eu': '1', 'Ga': '4'}
{'Ce': '8', 'Ru': '6'}
{'O': '11', 'U': '6'}
{'Bi': '6', 'Se': '6'}
{'B': '6', 'Ba': '1'}
{'Ga': '2', 'Th': '4'}
{'Te': '12', 'U': '8'}
{'Ni': '4', 'Si': '2'}
{'Nd': '1', 'Th': '3'}
{'Au': '1', 'Mg': '1'}
{'Pt': '4', 'Sn': '6'}
{'Mo': '4', 'S': '6'}
{'Be': '44', 'Mo': '2'}
{'Al': '2', 'Hf': '4'}
{'Ti': '1', 'Zn': '1'}
{'Mg': '16', 'Pd': '8'}
{'As': '4', 'Cr': '2'}
{'Sb': '4', 'Yb': '2'}
{'Hg': '1', 'Mg': '3'}
{'Au': '2', 'Ta': '6'}
{'O': '4', 'Ru': '2'}
{'Dy': '1', 'Mg': '3'}
{'Li': '13', 'Sn': '5'}
{'F': '6', 'Pd': '2'}
{'O': '10', 'U': '5'}
{'B': '2', 'Ho': '1'}
{'S': '2', 'V': '1'}
{'Co': '9', 'Dy': '3'}
{'F': '6', 'Ni': '2'}
{'Pb': '6', 'Sr': '4'}
{'Ir': '8', 'U': '8'}
{'Er': '3', 'La': '1'}
{'Au': '1', 'V': '3'}
{'Ge': '16', 'Yb': '20'}
{'Fe': '3', 'Si': '1'}
{'Al': '10', 'Nb': '20'}
{'Pd': '2', 'V': '6'}
{'Eu': '6', 'Ga': '4'}
{'Cu': '4', 'Sm': '4'}
{'Cu': '20', 'Sb': '6'}
{'Ni': '4', 'Y': '4'}
{'Mn': '1', 'Ni': '3'}
{'Au': '1', 'Dy':

{'Nb': '3', 'Si': '6'}
{'Cl': '6', 'Ti': '2'}
{'Ni': '34', 'Tm': '4'}
{'Er': '10', 'Ga': '6'}
{'Co': '4', 'Nd': '2'}
{'Cd': '3', 'Ce': '1'}
{'Pt': '2', 'Si': '1'}
{'Pt': '3', 'Sn': '1'}
{'Cu': '1', 'Pt': '1'}
{'Al': '4', 'Ho': '8'}
{'Dy': '2', 'Ni': '4'}
{'Rh': '3', 'Sc': '1'}
{'Bi': '6', 'Ce': '8'}
{'Ge': '1', 'U': '3'}
{'Hg': '1', 'Sc': '1'}
{'Ag': '2', 'In': '4'}
{'Al': '6', 'Zr': '10'}
{'O': '84', 'W': '32'}
{'Co': '1', 'Zn': '13'}
{'B': '4', 'V': '6'}
{'Nd': '1', 'Tl': '3'}
{'In': '3', 'Pd': '2'}
{'Rh': '4', 'Si': '4'}
{'In': '8', 'Ti': '6'}
{'Ag': '2', 'Tb': '1'}
{'Ru': '3', 'U': '1'}
{'Hf': '2', 'Mo': '4'}
{'Ir': '5', 'Th': '1'}
{'Pt': '2', 'Sb': '2'}
{'Au': '6', 'La': '2'}
{'Co': '12', 'Mg': '12'}
{'Rh': '2', 'Zn': '6'}
{'Ge': '10', 'Ho': '6'}
{'As': '2', 'Fe': '4'}
{'Cr': '4', 'Si': '4'}
{'Mn': '1', 'N': '1'}
{'Ca': '2', 'Zn': '26'}
{'C': '4', 'Mo': '8'}
{'Ta': '12', 'Te': '48'}
{'H': '18', 'Nd': '6'}
{'Ni': '3', 'Sn': '1'}
{'Ir': '6', 'Th': '14'}
{'Ho': '10', 'Si': '17'}
{'Er

{'Hg': '4', 'K': '2'}
{'Ge': '6', 'Tb': '2'}
{'Pd': '3', 'Pm': '1'}
{'Cs': '6', 'Hg': '40'}
{'Be': '17', 'Nb': '2'}
{'Se': '12', 'U': '8'}
{'Pt': '4', 'Te': '6'}
{'Yb': '2', 'Zn': '4'}
{'Ni': '4', 'Sc': '2'}
{'Mg': '18', 'Sn': '10'}
{'Mn': '11', 'Pd': '21'}
{'Ho': '6', 'Tm': '2'}
{'Nd': '3', 'Sm': '1'}
{'Ge': '16', 'Hf': '20'}
{'Cu': '1', 'Sc': '1'}
{'Gd': '1', 'In': '1'}
{'Rh': '3', 'Ta': '1'}
{'Ce': '16', 'Se': '24'}
{'Sm': '3', 'Tm': '1'}
{'Ba': '2', 'Be': '26'}
{'O': '24', 'W': '12'}
{'Ag': '4', 'La': '2'}
{'Te': '8', 'Tl': '8'}
{'Ga': '14', 'Ni': '6'}
{'Hg': '2', 'Yb': '1'}
{'Cd': '2', 'Ho': '1'}
{'Mn': '7', 'O': '12'}
{'H': '2', 'Pu': '1'}
{'Ce': '5', 'O': '9'}
{'Se': '1', 'Tm': '1'}
{'As': '6', 'Mo': '4'}
{'Ca': '2', 'Zn': '2'}
{'B': '2', 'Mg': '1'}
{'Ir': '18', 'Zr': '30'}
{'Rh': '5', 'Tb': '1'}
{'Ga': '2', 'Ti': '6'}
{'Cl': '4', 'Nd': '2'}
{'O': '12', 'Re': '4'}
{'Fe': '24', 'N': '9'}
{'Ce': '3', 'Ir': '9'}
{'C': '6', 'Li': '1'}
{'Be': '2', 'Hf': '1'}
{'Al': '6', 'Hf': '10'}
{

{'N': '2', 'Re': '4'}
{'Cd': '1', 'Rb': '3'}
{'Er': '2', 'Te': '4'}
{'Nb': '4', 'Sn': '8'}
{'Ni': '15', 'O': '16'}
{'Mg': '3', 'Sm': '1'}
{'Te': '1', 'Tm': '1'}
{'Sm': '1', 'Y': '3'}
{'Ge': '2', 'Nb': '6'}
{'Ge': '14', 'Nd': '8'}
{'Al': '4', 'Th': '2'}
{'Au': '4', 'P': '6'}
{'Cl': '8', 'U': '2'}
{'Sb': '1', 'Tb': '1'}
{'Er': '3', 'Pr': '1'}
{'Rh': '4', 'Si': '4'}
{'Hf': '1', 'Os': '1'}
{'Cd': '1', 'Sc': '1'}
{'Au': '6', 'Sm': '2'}
{'In': '1', 'Pr': '1'}
{'Cu': '1', 'Eu': '1'}
{'Ho': '1', 'In': '1'}
{'Pu': '10', 'Ru': '6'}
{'Hf': '6', 'Sb': '6'}
{'Pr': '1', 'Sm': '3'}
{'Pt': '16', 'Y': '20'}
{'Mo': '8', 'N': '8'}
{'S': '2', 'Ti': '1'}
{'Al': '2', 'Sc': '6'}
{'Hf': '8', 'Ni': '21'}
{'P': '4', 'Re': '8'}
{'Pr': '2', 'Pt': '4'}
{'Ge': '4', 'La': '4'}
{'Sb': '20', 'Sr': '22'}
{'Pr': '2', 'Pt': '2'}
{'Fe': '10', 'O': '11'}
{'Cu': '2', 'La': '1'}
{'C': '1', 'Hf': '1'}
{'F': '4', 'Pd': '2'}
{'Se': '8', 'Ti': '6'}
{'Er': '1', 'Te': '1'}
{'Co': '9', 'Er': '3'}
{'B': '16', 'Ru': '22'}
{'S': '4', 

{'O': '30', 'Ti': '19'}
{'Al': '4', 'Yb': '2'}
{'Al': '3', 'Pd': '2'}
{'Cd': '1', 'Dy': '1'}
{'Dy': '1', 'Hg': '1'}
{'Cl': '6', 'Pu': '2'}
{'Bi': '3', 'Sm': '1'}
{'Al': '3', 'Zr': '1'}
{'As': '1', 'Zr': '1'}
{'Ga': '3', 'Pd': '7'}
{'Dy': '4', 'Tl': '2'}
{'Br': '30', 'W': '6'}


# SubtaskD1.h: Calculations

Task: more detail analysis of Calculations

`print('\n\nMore detailed analysis of Calculations \n')`

List, stacked Histogram of Calculations types and the state it ended up finished, failed, exit codes, exit messages

more detail analysis of WorkChains

`print('\n\nMore detailed analysis of WorkChains \n')`

List,  stacked Histogram for each Workchain type and the state it ended up in finished, failed, exit codes, exit messages

In [83]:
def GetCalNode():
    exit_state = []
    exit_message = []
    index = []
    exit_state_digit = []
    for node, in CalcNode:
        print(str(node.process_state))
        print(node.pk)
        #print(str(node.exit_message))
        exit_state = exit_state + [str(node.process_state)]
        exit_message = exit_message + [str(node.exit_message)]
        index = index + [node.pk]
        if node.is_finished_ok:
            exit_state_digit = exit_state_digit + [1]
        else:
            exit_state_digit = exit_state_digit + [0]
    return exit_state,exit_message,index,exit_state_digit

def ShowCalNode(exit_state,exit_message,index,exit_state_digit):
    from bokeh.io import output_file, show
    from bokeh.models import ColumnDataSource
    from bokeh.plotting import figure
    from bokeh.io import output_notebook
    from bokeh.palettes import inferno

    output_file("ShowingCal.html")

    index = index
    exit_message = exit_message
    exit_state_string = exit_state
    exit_state_digit = exit_state_digit
      
    source = ColumnDataSource(data=dict(index=index, exit_state_digit=exit_state_digit,exit_message = exit_message,exit_state_string = exit_state_string, color=inferno(len(index))))
    
    TOOLTIPS = [
    ("Exit information", "@exit_message"),
    ("(x,y)", "($x, $y)"),
    ("Node index", "@index"),
    ("Node status", "@exit_state_string"),   
    ]

    p = figure( y_range=(0,1), x_range=(0,3000), plot_width=800, plot_height=800, title="CalcNode Information", tooltips=TOOLTIPS)
    #print('step figure done')
    p.vbar(x="index", top="exit_state_digit", bottom=0, width=1, color='color',  source=source)
    #print('step hbar done')
    
    output_notebook()
    p.xgrid.grid_line_color = None
    #p.legend = False
    show(p)
    
def GetWorkflowNode(Name):
    from aiida.orm import WorkflowNode
    from aiida.orm import QueryBuilder
    qb = QueryBuilder()
    qb.append(Name)
    WNode = qb.all()

    Newdict = {}
    for node, in WNode:
        if node.is_finished_ok:
            Newdict[node.node_type+'_succeed'] = Newdict.get(node.node_type+'_succeed',0) + 1
            Newdict[node.node_type+'_not_succeed'] = Newdict.get(node.node_type+'_not_succeed',0) + 0
        else:
            Newdict[node.node_type+'_not_succeed'] = Newdict.get(node.node_type+'_not_succeed',0) + 1
            Newdict[node.node_type+'_succeed'] = Newdict.get(node.node_type+'_succeed',0) + 0
    return Newdict
    

In [84]:
from aiida.orm import CalcJobNode
from aiida.orm import QueryBuilder
qb = QueryBuilder()
qb.append(CalcJobNode)
CalcNode = qb.all()

exit_state,exit_message,index,exit_state_digit = GetCalNode()


ProcessState.FINISHED
4979
ProcessState.FINISHED
4966
ProcessState.FINISHED
4570
ProcessState.FINISHED
4110
ProcessState.FINISHED
4069
ProcessState.FINISHED
3424
ProcessState.FINISHED
3345
ProcessState.FINISHED
3279
ProcessState.FINISHED
2680
ProcessState.FINISHED
2621
ProcessState.FINISHED
2552
ProcessState.FINISHED
2383
ProcessState.FINISHED
2375
ProcessState.FINISHED
1973
ProcessState.FINISHED
1547
ProcessState.FINISHED
1489
ProcessState.FINISHED
1361
ProcessState.FINISHED
845
ProcessState.FINISHED
814
ProcessState.FINISHED
782
ProcessState.FINISHED
302
ProcessState.FINISHED
241
ProcessState.FINISHED
240
ProcessState.FINISHED
239
ProcessState.FINISHED
238
ProcessState.FINISHED
237
ProcessState.FINISHED
236
ProcessState.FINISHED
235
ProcessState.FINISHED
234
ProcessState.FINISHED
233
ProcessState.FINISHED
232
ProcessState.FINISHED
231
ProcessState.FINISHED
230
ProcessState.FINISHED
228
ProcessState.FINISHED
227
ProcessState.FINISHED
226
ProcessState.FINISHED
225
ProcessState.FINISHED

In [43]:
ShowCalNode(exit_state,exit_message,index,exit_state_digit)

Loading BokehJS ...

In [49]:
Newdict1 = GetWorkflowNode(WorkflowNode)
Newdict2 = GetWorkflowNode(CalcJobNode)
print(Newdict1)
print(Newdict2)

{'process.workflow.workchain.WorkChainNode._not_succeed': 28, 'process.workflow.workchain.WorkChainNode._succeed': 0, 'process.workflow.workfunction.WorkFunctionNode._not_succeed': 49, 'process.workflow.workfunction.WorkFunctionNode._succeed': 0}
{'process.calculation.calcjob.CalcJobNode._succeed': 43, 'process.calculation.calcjob.CalcJobNode._not_succeed': 0}


In [52]:
def ShowWorkflow(WorkflowDict):
    from bokeh.io import output_file, show
    from bokeh.models import ColumnDataSource,HoverTool
    from bokeh.plotting import figure
    from bokeh.io import output_notebook
    from bokeh.palettes import inferno

    output_file("ShowingCal.html")

    index = list(WorkflowDict.keys())
    counts = list(WorkflowDict.values())
    #exit_message = exit_message
    #exit_state_string = exit_state
    #exit_state_digit = exit_state_digit
      
    source = ColumnDataSource(data=dict(index=index, counts=counts, color=inferno(len(index))))
    
    TOOLTIPS = [
    ("Node number", "@counts"),
    ("(x,y)", "($x, $y)"),
    ("Node status", "@index"),   
    ]
   
    HT = HoverTool(
    tooltips=TOOLTIPS,

    mode='vline'
    )
    
    p = figure( y_range=(0,50), x_range=index, plot_width=800, plot_height=800, title="Process Node Information",tools = [HoverTool(mode='vline')],tooltips=TOOLTIPS)
    #print('step figure done')
    p.vbar(x="index", top="counts", bottom=0, width=1, color='color',  source=source)
    #print('step hbar done')
    
    output_notebook()
    p.xgrid.grid_line_color = None
    #p.legend = False
    show(p)

ShowWorkflow(Newdict1)
ShowWorkflow(Newdict2)

Loading BokehJS ...

Loading BokehJS ...

# SubtaskD1.i: Provenance

Task: Database and provenance health: display the number of nodes who have no incomming and outgoing links, no incomming links (any number outgoing), and no outgoing links (any number incomming)

In [53]:
def Count_In_Out():
    from aiida.orm import QueryBuilder
    qb = QueryBuilder()
    qb.append(Node)
    Nodes = qb.all()
    Namelist = ['No_Incoming','No_Outgoing','No_In&Out']

    Mydict = {}
    for n, in Nodes:
        Incoingflag,Outgoingflag = False,False
        if(n.get_incoming().all_nodes() == []):
            Incoming_flag = True
            Mydict[Namelist[0]] = Mydict.get(Namelist[0],0)+1
        if(n.get_outgoing().all_nodes() == []):
            Outgoingflag = True
            Mydict[Namelist[1]] = Mydict.get(Namelist[1],0)+1
        if(Incoming_flag and Outgoingflag):
            Mydict[Namelist[2]] = Mydict.get(Namelist[2],0)+1

    return Mydict

def Show_In_Out(Mydict):
    from bokeh.io import output_file, show
    from bokeh.models import ColumnDataSource,HoverTool
    from bokeh.plotting import figure
    from bokeh.io import output_notebook
    from bokeh.palettes import Category20

    output_file("Show_In_Out.html")

    index = list(Mydict.keys())
    counts = list(Mydict.values())
      
    source = ColumnDataSource(data=dict(index=index, counts=counts, color=Category20[len(index)]))
    
    TOOLTIPS = [
    ("Node number", "@counts"),
    ("(x,y)", "($x, $y)"),
    ("Node status", "@index"),   
    ]
   
    HT = HoverTool(
    tooltips=TOOLTIPS,

    mode='vline'
    )
    
    p = figure( y_range=(0,6000), x_range=index, plot_width=800, plot_height=800, title="CalcNode Information",tools = [HoverTool(mode='vline')],tooltips=TOOLTIPS)
    #print('step figure done')
    p.vbar(x="index", top="counts", bottom=0, width=1, color='color',  source=source)
    #print('step hbar done')
    
    output_notebook()
    p.xgrid.grid_line_color = None
    #p.legend = False
    show(p)

In [54]:
Mydict = Count_In_Out()
print(Mydict)

{'No_Incoming': 5516, 'No_Outgoing': 5504, 'No_In&Out': 5504}


In [55]:
Show_In_Out(Mydict)

Loading BokehJS ...

0